In [1]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
import pickle
from pydantic import BaseModel
import tiktoken
import os.path
import pprint

In [2]:
#https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

CHAT_GPT_MODEL = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model("gpt-4")

In [3]:
themes_cache = "cache/themes.pkl"

In [4]:
from os.path import expanduser
load_dotenv(os.path.join(expanduser("~"), ".env"))

True

In [5]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [6]:
data_file = "data_input.xlsx"
times = pd.read_excel(data_file, sheet_name="Data", index_col="ID")
demog = pd.read_excel(data_file, sheet_name="Demographic")

In [7]:
times.columns = [
    "start_time",
    "completion_time",
    "star_rating",
    "txt_what_liked",
    "txt_what_not_liked",
    "txt_do_to_improve",
    "txt_anything_else",
    "recommend_likelihood",
    "ref_num",
    "person_id",
    "start_group",
    "gender",
    "age_group",
    "country",
    "province",
    "number_finished",
    "reg_day",
    "reg_hour",
    "PPA",
    "has_result",
]

demog.columns = [
    "person_id",
    "start_group",
    "gender",
    "age_group",
    "country",
    "province",
    "number_finished",
    "reg_day",
    "reg_hour",
    "PPA",
    "has_result",
]

In [8]:
all_data = pd.merge(
    times, demog, on="person_id", how="left", suffixes=(None, "_demog")
).convert_dtypes()

all_data.drop(
    columns=[
        "start_group",
        "gender",
        "age_group",
        "country",
        "province",
        "number_finished",
        "reg_day",
        "reg_hour",
        "PPA",
        "has_result",
    ],
    inplace=True,
)

In [9]:
all_data = all_data.astype(
    {
        "start_group_demog": "category",
        "gender_demog": "category",
        "age_group_demog": "category",
        "country_demog": "category",
        "province_demog": "category",
    }
)

In [10]:
# Text cleanup
no_answer_text = "(none)"

txt_cols = [
    "txt_what_liked",
    "txt_what_not_liked",
    "txt_do_to_improve",
    "txt_anything_else",
]

all_data[txt_cols] = all_data[txt_cols].fillna(no_answer_text)

search_pattern = "|".join(["NA", "Na"])

for col in txt_cols:
    all_data[col] = all_data[col].str.replace(search_pattern, no_answer_text, regex=True)
    all_data[col] = all_data[col].replace("", no_answer_text)
    all_data[col] = all_data[col].str.replace(r"[^a-zA-Z ]", "", regex=True) #not needed
    all_data[col] = all_data[col].str.replace("\n", " ") #remove line breaks

In [11]:
# play with tiktoken
for col in txt_cols:
    test_text = " ".join(all_data[col].to_list())
    tokens = encoding.encode(test_text)
    print(col, len(tokens))

txt_what_liked 62476
txt_what_not_liked 96681
txt_do_to_improve 78861
txt_anything_else 58871


In [12]:
len_times = len(times)
lost_records = len(all_data) - len_times
lost_records_percent = lost_records / len_times
print(
    f"Data with no demographic records: {lost_records} rows, {lost_records_percent:.2%} of {len_times} total records."
)

Data with no demographic records: 55 rows, 0.95% of 5784 total records.


In [13]:
# GPT call for themes

def GPT_get_themes(responses):
    # OpenAI API call to summarize the text

    class Theme(BaseModel):
        theme_id: int
        theme_text: str

    class AllThemes(BaseModel):
        themes: list[Theme]

    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"""The following is a list of responses to a single question in a market research survey. 
                
                Create an overall list of themes extracted from all answers. There shoud be at most 20 themes, 
                and they should have mninimal overlap. Each theme should be a maximum of 20 words.
                Each theme will have an index called theme_id and the theme itself as theme_text.
                Return all the themes ina list called 'themes'
                
                Here are the responses{responses}""",
            },
        ]

    completion   = client.beta.chat.completions.parse(
        model=CHAT_GPT_MODEL,
        temperature = 0.3,
        messages=messages,
        response_format=AllThemes,
    )

    # Extract the summary from the response
    return completion.choices[0].message.parsed
  

In [24]:
# GPT call for theme matching
# https://platform.openai.com/docs/guides/structured-outputs
def GPT_match_themes(themes, responses):

    class EachAnswer(BaseModel):
        response_id: int
        theme_id: int
        theme_text: str

    class AllAnswers(BaseModel):
        classifications: list[EachAnswer]

    messages = [
            {
                "role": "system",
                "content": "You are an assistant for matching human responses to a survey to pre-existing themes.",
            },
            {
                "role": "user",
                "content": f"""I have a list of themes summarised over some responses to a survey question. The themes represent common topics found in the resposnes.
        Here are the themes: {themes}.
        
        I will give you the responses used to generate the themes. Each response has its own id called response_id.

        For each response, I want you to identify which one of the themes most closely represents the response.
        Return the answers in the object EachAnswer.
    
        Return the original response_id, the theme_id and theme_text of the most representative theme.
        
        However, if the response text is {no_answer_text}, there will be no theme.
        In this case. return the original response_id, 0 as the theme index and "no theme" as the theme text.

        Return all the EachAnswer objects in a final object called AllAnswers

        Here are the responses {responses}:""",
            },
        ]

    completion = client.beta.chat.completions.parse(
        model=CHAT_GPT_MODEL,
        temperature=0.1,
        messages=messages,
        response_format=AllAnswers,
    )

    result = completion.choices[0].message.parsed

    pprint.pp(result.classifications)

    return result

In [15]:
# GPT call to summarise inputs

def GPT_summarize_responses(inputs):
    # OpenAI API call to summarize the text
    response = client.chat.completions.create(
        model=CHAT_GPT_MODEL,  # Specify the model you want to use
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant for summarising survey responses.",
            },
            {
                "role": "user",
                "content": f"""You will be given a list of responses to a question in a survey. Your job is to extract key themes from the responses.
                Ignore any responses that are very short, are empty, or have the text {no_answer_text}
                Each theme should have a headline, followed by an explanatory paragraph. For each theme, provide from 1 to 3 verbatim quotes to illustrate the theme along side the explanatory paragraph.
                Don't provide any duplicated verbatim quotes.
                Sort the themes by their decreasing frequency of appearance. At the end, be sure to say which was the most commonly seen theme, and which was the least commonly seen.

                Here are your inputs:\n\n{inputs}""",
            },
        ],
    )

    # Extract the summary from the response
    summary = response.choices[0].message.content.strip()
    return summary

In [16]:
txt_cols_to_classify = [
    "txt_what_liked",
    "txt_what_not_liked",
    "txt_do_to_improve",
    "txt_anything_else",
]

txt_cols_to_summarise = [
    "txt_anything_else",
]

In [17]:
# Get the themes

def extract_themes(dataframe, columns):

    if os.path.isfile(themes_cache):

        with open(themes_cache, "rb") as picklefile:
            themes_dict = pickle.load(picklefile)

        return themes_dict

    themes_dict = {}

    for column in columns:

        print(f"Column: {column}")
        response_list = " ".join(dataframe[column].to_list())
        result = GPT_get_themes(response_list)
        theme_list = [(theme.theme_id, theme.theme_text) for theme in result.themes]
        themes_dict[column] = theme_list

    with open(themes_cache, "wb") as picklefile:
        pickle.dump(themes_dict, picklefile)

    return themes_dict

In [18]:
# Create themes
themes_dict = extract_themes(all_data, txt_cols_to_classify)

In [26]:
# Classify the columns

def classify_text_column(orig_dataframe, columns):

    # Create a deep copy with indices to return for pickling
    dataframe = orig_dataframe.copy()

    with open(themes_cache, "rb") as picklefile:
        themes_dict = pickle.load(picklefile)

    for column in columns:

        print(f"Column: {column}")
        response_list = dataframe[column].to_list()
        expected_num_outputs = len(response_list)

        responses_input = [
            f"{idx} {txt}"
            for idx, txt in zip(dataframe[column].index, dataframe[column])
        ]
        # print(responses_input)

        expected_num_outputs = len(response_list)

        # responses_input = "\n".join(response_list)
        responses_input_for_output = "".join([f"{i}. {response} \n" for i, response in enumerate(response_list)])

        # responses_input = str(response_list)

        with open(f"outputs/A {column} responses_input.txt", "w") as text_file:
            text_file.write(responses_input_for_output)

        themes_for_input = themes_dict[column]

        # print(themes_for_input)

        response = GPT_match_themes(themes_for_input, responses_input)

        actual_num_outputs = len(response.classifications)

        #pprint.pp(response.classifications)

        print(
            f"Expected, actual responses: {expected_num_outputs}, {actual_num_outputs}"
        )

        with open(f"outputs/B {column} classifications.txt", "w") as text_file:
            for el in response.classifications:
                text_file.write(f"{el.response_id} {el.theme_id} {el.theme_text} \n")

        classified_themes = [resp.theme_text for resp in response.classifications]

        column_index = dataframe.columns.get_loc(column)
        new_name = f'{column}_theme'

        try:
            dataframe.insert(column_index + 1, new_name, classified_themes)
            print(f'New column {new_name} inserted')
            # print(dataframe)

        except:
            print("failed to insert column")
            pass

        # dataframe.to_pickle(classifications_cache)

    return dataframe

In [32]:
# Loop through chunks

working_data = all_data.copy()
total_records = len(working_data)

print("Total Records", total_records)
chunk_size = 60
working_txt_cols_to_classify = txt_cols_to_classify#[0:1]
chunks_folder = "df_chunks/"

start, end = 0, chunk_size

# Delete Previous Chunks
for filename in os.listdir(chunks_folder):
    os.remove(f"{chunks_folder}{filename}")

counter = 1

while start != total_records:

    print("Start, end = ", start, end)

    current_chunk = working_data[start:end].copy()
    #pprint.pp(current_chunk["start_time"])

    classifications = classify_text_column(
        current_chunk,
        working_txt_cols_to_classify,
    )

    # print(classifications['txt_what_liked'])

    output_file = f"{chunks_folder}df {counter:04d} {start:04d}-{end:04d}.pkl"

    with open(output_file, "wb") as f:
        classifications.to_pickle(f)

    remaining_records = total_records - end
    print("Remaining", remaining_records)

    if remaining_records > chunk_size:

        print("banana", start, chunk_size, total_records)
        start = end
        end = start + chunk_size

    else:
        print("xxx", start, end, total_records, remaining_records)
        start = end
        end = start + remaining_records

    print("New start, end", start, end)

    print()

    counter += 1

# classifications
# working_data

Total Records 5839
Start, end =  0 60
Column: txt_what_liked
[EachAnswer(response_id=0, theme_id=0, theme_text='no theme'),
 EachAnswer(response_id=1, theme_id=18, theme_text='Impressive organization for such a large event.'),
 EachAnswer(response_id=2, theme_id=18, theme_text='Impressive organization for such a large event.'),
 EachAnswer(response_id=3, theme_id=8, theme_text='Full road closures ensuring cyclist safety.'),
 EachAnswer(response_id=4, theme_id=1, theme_text='Well organized event with efficient logistics.'),
 EachAnswer(response_id=5, theme_id=0, theme_text='no theme'),
 EachAnswer(response_id=6, theme_id=2, theme_text='Spectacular scenic route around the Cape Peninsula.'),
 EachAnswer(response_id=7, theme_id=0, theme_text='no theme'),
 EachAnswer(response_id=8, theme_id=8, theme_text='Full road closures ensuring cyclist safety.'),
 EachAnswer(response_id=9, theme_id=2, theme_text='Spectacular scenic route around the Cape Peninsula.'),
 EachAnswer(response_id=10, theme_i

In [33]:
df_chunks = []

print('Reconstituting...')
for filename in os.listdir(chunks_folder):

    temp_df = pd.read_pickle(f'{chunks_folder}{filename}')
    print(filename, len(temp_df))
    df_chunks.append(temp_df)

reconstituted = pd.concat(df_chunks)
reconstituted.sort_index(inplace=True)

reconstituted.to_pickle('outputs/final_reconstituted.pkl')
print(len(reconstituted))
reconstituted

Reconstituting...
df 0063 3720-3780.pkl 60
df 0043 2520-2580.pkl 60
df 0023 1320-1380.pkl 60
df 0003 0120-0180.pkl 60
df 0072 4260-4320.pkl 60
df 0047 2760-2820.pkl 60
df 0036 2100-2160.pkl 60
df 0083 4920-4980.pkl 60
df 0015 0840-0900.pkl 60
df 0037 2160-2220.pkl 60
df 0004 0180-0240.pkl 60
df 0040 2340-2400.pkl 60
df 0052 3060-3120.pkl 60
df 0009 0480-0540.pkl 60
df 0029 1680-1740.pkl 60
df 0085 5040-5100.pkl 60
df 0044 2580-2640.pkl 60
df 0091 5400-5460.pkl 60
df 0014 0780-0840.pkl 60
df 0024 1380-1440.pkl 60
df 0032 1860-1920.pkl 60
df 0070 4140-4200.pkl 60
df 0048 2820-2880.pkl 60
df 0025 1440-1500.pkl 60
df 0089 5280-5340.pkl 60
df 0055 3240-3300.pkl 60
df 0080 4740-4800.pkl 60
df 0017 0960-1020.pkl 60
df 0090 5340-5400.pkl 60
df 0094 5580-5640.pkl 60
df 0026 1500-1560.pkl 60
df 0010 0540-0600.pkl 60
df 0084 4980-5040.pkl 60
df 0097 5760-5820.pkl 60
df 0056 3300-3360.pkl 60
df 0027 1560-1620.pkl 60
df 0093 5520-5580.pkl 60
df 0039 2280-2340.pkl 60
df 0016 0900-0960.pkl 60
df 0087

,start_time,completion_time,star_rating,txt_what_liked,txt_what_liked_theme,txt_what_not_liked,txt_what_not_liked_theme,txt_do_to_improve,txt_do_to_improve_theme,txt_anything_else,...,start_group_demog,gender_demog,age_group_demog,country_demog,province_demog,number_finished_demog,reg_day_demog,reg_hour_demog,PPA_demog,has_result_demog
0,2024-03-20 13:45:57,2024-03-20 13:47:45,<NA>,none,no theme,none,no theme,none,no theme,none,...,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2024-03-20 14:00:18,2024-03-20 14:02:21,5,The Team who organise it,Impressive organization for such a large event.,Nothing comes to mind,no theme,Alp du Hez experience at the top of Alp du Sui...,Encourage more crowd support and entertainment...,brilliant,...,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2024-03-20 14:27:17,2024-03-20 14:28:30,4,well organised great helpers,Impressive organization for such a large event.,start a little too late,Lack of clear signage at the start and finish.,start earlier,Implement a rolling start to reduce bottleneck...,much better than KMs,...,9E,2,45-49,South Africa,Western Cape,3,8,11,<NA>,1
3,2024-03-20 14:27:17,2024-03-20 14:28:48,5,Road closures,Full road closures ensuring cyclist safety.,It is getting dangerous with larger groups,Large start groups causing safety concerns.,Rider safety in large groups,Ensure better road conditions along the race r...,Nope,...,#,2,40-44,South Africa,Western Cape,20,8,13,<NA>,1
4,2024-03-20 14:27:32,2024-03-20 14:28:52,5,Well organized event and great route,Well organized event with efficient logistics.,Think its starting to get quite pricey,High entry fees compared to value received.,Na,no theme,Na,...,8B,2,35-39,South Africa,Western Cape,3,7,12,<NA>,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5834,2024-04-08 16:06:24,2024-04-08 16:25:14,4,Nice Scenery Clean and good road Support Far bars,Spectacular scenic route around the Cape Penin...,Parking is always a problem Have to be really...,Lack of food and snacks at water stations.,It seems we pay more and get less No usable vo...,Improve the quality and variety of items in th...,It is still my best SA cycle event so far But ...,...,Dh,1,55-59,South Africa,Western Cape,3,9,11,<NA>,1
5835,2024-04-09 08:00:24,2024-04-09 08:01:21,5,Everything,no theme,No free coffees at the end,Lack of food and snacks at water stations.,Free coffees at the end,no theme,Loved it,...,7C,2,40-44,South Africa,Western Cape,6,8,15,<NA>,1
5836,2024-04-09 08:45:35,2024-04-09 08:53:33,5,Wonderful route to ride on closed roads Except...,Full road closures ensuring cyclist safety.,Strange seeding anomalies and Ebikes as part o...,Confusing communication regarding race rules a...,In time there needs to be dedicated Ebike events,Create a separate start group for ebikes to av...,none,...,4A,2,60-64,South Africa,Western Cape,6,9,13,<NA>,1
5837,2024-04-09 09:56:55,2024-04-09 09:59:45,5,Atmostphere,Great atmosphere and camaraderie among partici...,wind,Windy conditions affecting race experience.,more castle lites at the beer tent,no theme,no,...,6A,2,65-69,South Africa,Limpopo,<NA>,9,11,<NA>,1
